In [101]:
from bs4 import BeautifulSoup as soup  # HTML data structure
#This allows us to make requests 
from urllib.request import urlopen as uReq  # Web client


<h2> First we have to get all of the URL links to game log pages </h2>

In [102]:
year = 2020

#Here we enter the URL we are trying to web scrape from 
page_url = "https://www.basketball-reference.com/leagues/NBA_" + str(year) + "_per_game.html"

#this basically downloads the webpage into an object
uClient = uReq(page_url)
# parses html into a soup data structure to traverse html
# as if it were a json data type.
page_soup = soup(uClient.read(), "html.parser")
uClient.close()


In [104]:

name_containers = page_soup.findAll("td", {"class": "left"})

urls = []

isName = True
for c in name_containers: 
    #the names rotate from player name to team name so the flag changes each time
    if isName:
        urls.append(c.find('a')['href'])
        isName = False 
    else:
        isName = True
        continue

urls = [url.replace('.html','/gamelog/2020') for url in urls]


In [105]:
for url in urls:
    page_url = "https://www.basketball-reference.com" + str(url)

    #this basically downloads the webpage into an object
    uClient = uReq(page_url)
    # parses html into a soup data structure to traverse html
    # as if it were a json data type.
    page_soup = soup(uClient.read(), "html.parser")
    uClient.close()

    #Getting player name 
    player_name = page_soup.find("title").text
    player_name = player_name.replace(' ' +str(year-1) + '-' + str(year%100) +' Game Log | Basketball-Reference.com',"")
    player_name = player_name.replace(' ','_')
    
    print(player_name)
    
    #getting all of the rows
    rows = page_soup.findAll("tr")

    #seperating games played vs games missed
    games_missed = []
    games = []

    #sorting games played from games missed 
    for row in rows:

        if 'data-stat="reason"' in str(row): 
            games_missed.append(row)
        elif '<tr id="pgl_basic' in str(row):
            games.append(row)

    #setting arrays in to store data 
    game_number = []
    date = []
    age = []
    team = []
    home_road = []
    opponent = []
    game_result = []
    games_started = []
    minutes_played = []
    fg_made = []
    fg_attempts = []
    fg_percent = []
    three_pt_made = []
    three_pt_attempts = []
    three_pt_percent = []
    ft_made = []
    ft_attempts = []
    ft_percent = []
    orb = []
    drb =[]
    trb = []
    ast =[]
    stl=[]
    blk= []
    tov = []
    pf =[]
    pts=[]
    game_score = []
    plus_minus = []
    
    #appending the values from bbref to these lists
    for game in games:

        game_number.append(game.find("th",{"data-stat":"ranker"}).text)
        date.append(game.find("td",{"data-stat":"date_game"}).text)
        age.append(game.find("td",{"data-stat":"age"}).text)
        team.append(game.find("td",{"data-stat":"team_id"}).text)
        home_road.append(game.find("td",{"data-stat":"game_location"}).text)
        opponent.append(game.find("td",{"data-stat":"opp_id"}).text)
        game_result.append(game.find("td",{"data-stat":"game_result"}).text)
        games_started.append(game.find("td",{"data-stat":"gs"}).text)
        minutes_played.append(game.find("td",{"data-stat":"mp"}).text)
        fg_made.append(game.find("td",{"data-stat":"fg"}).text)
        fg_attempts.append(game.find("td",{"data-stat":"fga"}).text)
        fg_percent.append(game.find("td",{"data-stat":"fg_pct"}).text)
        three_pt_made.append(game.find("td",{"data-stat":"fg3"}).text)
        three_pt_attempts.append(game.find("td",{"data-stat":"fg3a"}).text)
        three_pt_percent.append(game.find("td",{"data-stat":"fg3_pct"}).text)
        ft_made.append(game.find("td",{"data-stat":"ft"}).text)
        ft_attempts.append(game.find("td",{"data-stat":"fta"}).text)
        ft_percent.append(game.find("td",{"data-stat":"ft_pct"}).text)
        orb.append(game.find("td",{"data-stat":"orb"}).text)
        drb.append(game.find("td",{"data-stat":"drb"}).text)
        trb.append(game.find("td",{"data-stat":"trb"}).text)
        ast.append(game.find("td",{"data-stat":"ast"}).text)
        stl.append(game.find("td",{"data-stat":"stl"}).text)
        blk.append(game.find("td",{"data-stat":"blk"}).text)
        tov.append(game.find("td",{"data-stat":"tov"}).text)
        pf.append(game.find("td",{"data-stat":"pf"}).text)
        pts.append(game.find("td",{"data-stat":"pts"}).text)
        game_score.append(game.find("td",{"data-stat":"game_score"}).text)
        plus_minus.append(game.find("td",{"data-stat":"plus_minus"}).text)


    out_filename = "csv_files/"+ str(player_name) + "_game_log_" + str(year) + ".csv"
    headers = "G,DATE,AGE,TEAM,WHERE,OP,RES,GS,MP,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-\n"

    f = open(out_filename, "w")
    f.write(headers)
    
    #write these values to the file 
    for i in range(0,len(games)):
        f.write(game_number[i]+ ", " + date[i] + ", " + age[i] + "," + team[i] + "," +\
                home_road[i]+ ", " + opponent[i] + ", " +  game_result[i] + ", " +  games_started[i] + ", " +\
                minutes_played[i] + "," + fg_made[i] + "," +\
                fg_attempts[i]+ ", " + fg_percent[i] + ", " + three_pt_made[i] + "," + three_pt_attempts[i] + "," +\
                three_pt_percent [i]+ "," + ft_made[i] + "," +  ft_attempts[i] + "," +\
                ft_percent[i]+ ", " + orb[i] + "," + drb[i] + "," +  trb[i] + "," + ast[i]+ ", " + stl[i] + ", " +\
                blk[i] + "," + tov[i] + "," + pf[i]+ ", " + pts[i] + ", " + game_score[i], + ", " + plus_minus[i] + "\n")

    f.close()



Steven_Adams
Bam_Adebayo
LaMarcus_Aldridge
Nickeil_Alexander-Walker
Grayson_Allen
Jarrett_Allen
Kadeem_Allen
Al-Farouq_Aminu
Justin_Anderson
Kyle_Anderson
Ryan_Anderson
Giannis_Antetokounmpo
Kostas_Antetokounmpo
Thanasis_Antetokounmpo
Carmelo_Anthony
OG_Anunoby
Ryan_Arcidiacono
Trevor_Ariza
Trevor_Ariza
Trevor_Ariza
D.J._Augustin
Deandre_Ayton
Dwayne_Bacon
Marvin_Bagley
Lonzo_Ball
Mohamed_Bamba
J.J._Barea
Harrison_Barnes
RJ_Barrett
Will_Barton
Keita_Bates-Diop
Keita_Bates-Diop
Keita_Bates-Diop
Nicolas_Batum
Aron_Baynes
Kent_Bazemore
Kent_Bazemore
Kent_Bazemore
Darius_Bazley
Bradley_Beal
Malik_Beasley
Malik_Beasley
Malik_Beasley
Marco_Belinelli
Jordan_Bell
Jordan_Bell
Jordan_Bell
DeAndre'_Bembry
Dragan_Bender
Dragan_Bender
Dragan_Bender
Dāvis_Bertāns
Patrick_Beverley
Khem_Birch
Goga_Bitadze
Bismack_Biyombo
Nemanja_Bjelica
Eric_Bledsoe
Bogdan_Bogdanović
Bojan_Bogdanović
Jonah_Bolden
Jonah_Bolden
Jonah_Bolden
Marques_Bolden
Jordan_Bone
Isaac_Bonga
Devin_Booker
Chris_Boucher
Brian_Bowen
Ky

Tyrone_Wallace
Taurean_Waller-Prince
Derrick_Walton
Derrick_Walton
Derrick_Walton
Brad_Wanamaker
T.J._Warren
P.J._Washington
Yuta_Watanabe
Tremont_Waters
Paul_Watson
Paul_Watson
Paul_Watson
Quinndary_Weatherspoon
Russell_Westbrook
Coby_White
Derrick_White
Hassan_Whiteside
Andrew_Wiggins
Andrew_Wiggins
Andrew_Wiggins
Grant_Williams
Johnathan_Williams
Kenrich_Williams
Lou_Williams
Marvin_Williams
Marvin_Williams
Marvin_Williams
Robert_Williams
Nigel_Williams-Goss
Zion_Williamson
D.J._Wilson
Justise_Winslow
Christian_Wood
Delon_Wright
Justin_Wright-Foreman
Thaddeus_Young
Trae_Young
Cody_Zeller
Ante_Žižić
Ivica_Zubac
